In [3]:
from collections import namedtuple
import os, sys
import urllib.request as request
from zipfile import ZipFile
from deepClassifier.constant import *
from deepClassifier.utils.common import read_yaml, create_directories

In [2]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    'root_dir',
    'source_URL',
    'local_data_file',
    'unzip_dir',
])

In [ ]:
class ConfigurationManage:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config
        

In [ ]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
    
    def _preprocess(self, zf, filepath, working_dir):
        pass
    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file,mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for file in updated_list_of_files:
                self._preprocess(zf, file, self.config.unzip_dir)
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ('Cat' in f or 'Dog' in f)]
            
    
    

In [ ]:
from dataclasses import dataclass

@dataclass()
class DataIngestionConfig_data:
    root_dir: Path
    source_URL: str 
    local_data_file: Path
    unzip_dir: Path